#**Prétraitement des données**

* Normalisation des pixels
* Redimensionnement des images
* Augmentation des données



##Normalisation des pixels

In [ ]:
import cv2
import os
import numpy as np

def verifier_normalisation_images(input_folder):
  # Liste des images
  image_files = os.listdir(input_folder)
  #initialiser à True
  all_images_normalized = True
  # vérifier chacune des images
  for file in image_files:
      input_path = os.path.join(input_folder, file)
      # Charger l'image en format RGB
      image = cv2.imread(input_path, cv2.IMREAD_COLOR) / 255.0
      # Vérifier si l'image est normalisée
      if not verifier_normalisation(image):
          print(f"L'image {file} n'est pas normalisée.")
          all_images_normalized = False

  if all_images_normalized:
      print("Toutes les images du dossier sont normalisées.")
  else:
      print("Au moins une image dans le dossier n'est pas normalisée.")

def verifier_normalisation(image):
    # Vérifier les valeurs minimales et maximales pour chaque canal
    min_values = np.min(image, axis=(0, 1))
    max_values = np.max(image, axis=(0, 1))
    # Vérifier si les valeurs sont dans la plage [0, 1]
    normalized = np.all((min_values >= 0) & (max_values <= 1))
    return normalized

if __name__ == "__main__":
    # Spécifier le dossier d'entrée contenant les images
    input_folder = "/content/Images"

    # Faire appel à la fonction pour vérifier si toutes les images du dossier sont normalisées
    verifier_normalisation_images(input_folder)

Toutes les images du dossier sont normalisées.


##Redimensionement des images

Redimmenssioner les images du dataset original à la taille (800,1200)

In [ ]:
from PIL import Image
import os

def redimensionner_images(dossier_entree, dossier_sortie, nouvelle_taille=(800, 1200)):
    # Créer le dossier de sortie s'il n'existe pas
    if not os.path.exists(dossier_sortie):
        os.makedirs(dossier_sortie)

    # Parcourir toutes les images du dossier d'entrée
    for fichier in os.listdir(dossier_entree):
        chemin_image_entree = os.path.join(dossier_entree, fichier)

        # Vérifier si le fichier est une image
        if fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            # Charger l'image
            image = Image.open(chemin_image_entree)

            # Redimensionner l'image
            nouvelle_image = image.resize(nouvelle_taille)

            # Enregistrer l'image redimensionnée dans le dossier de sortie
            chemin_image_sortie = os.path.join(dossier_sortie, fichier)
            nouvelle_image.save(chemin_image_sortie)

# Dossiers d'entrée et de sortie
dossier_entree_images = "/content/drive/MyDrive/Images(original)"
dossier_sortie_images = "/content/Images_redim/"

# Redimensionner les images
redimensionner_images(dossier_entree_images, dossier_sortie_images)
print("Redimensionnement des images terminé.")

Redimensionnement des images terminé.


**Télécharger les images redimmensionées localement**

In [ ]:
from google.colab import files

# Télécharger le dossier en tant qu'archive ZIP
!zip -r Images_redim.zip Images_redim/

# Télécharger l'archive ZIP
files.download('Images_redim.zip')

  adding: Images_redim/ (stored 0%)
  adding: Images_redim/436954557.jpg (deflated 1%)
  adding: Images_redim/436959339.jpg (deflated 4%)
  adding: Images_redim/436953154.jpg (deflated 2%)
  adding: Images_redim/436952755.jpg (deflated 1%)
  adding: Images_redim/436952581.jpg (deflated 3%)
  adding: Images_redim/436963111.jpg (deflated 1%)
  adding: Images_redim/436963249.jpg (deflated 1%)
  adding: Images_redim/436954783.jpg (deflated 1%)
  adding: Images_redim/436951492.jpg (deflated 3%)
  adding: Images_redim/436955748.jpg (deflated 1%)
  adding: Images_redim/436964616.jpg (deflated 3%)
  adding: Images_redim/436963199.jpg (deflated 0%)
  adding: Images_redim/436968973.jpg (deflated 3%)
  adding: Images_redim/436961164.jpg (deflated 3%)
  adding: Images_redim/436962492.jpg (deflated 1%)
  adding: Images_redim/436968715.jpg (deflated 0%)
  adding: Images_redim/436955619.jpg (deflated 1%)
  adding: Images_redim/436958736.jpg (deflated 1%)
  adding: Images_redim/436955760.jpg (deflated

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Augmentation des données

In [4]:
import os
import cv2
import numpy as np

# Chemin vers le dossier contenant les images originales
images_folder = "/content/drive/MyDrive/Images"

# Chemin vers le dossier contenant les annotations YOLO originales
annotations_folder = "/content/drive/MyDrive/Annotations"

# Chemin vers le dossier où vous souhaitez enregistrer les nouvelles images et annotations
output_folder = "/content/drive/MyDrive/Augmente"

# Angles de rotation en degrés
rotation_angles = [0, 90, 180, 270]

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Parcourir toutes les images du dossier
for image_name in os.listdir(images_folder):
    if image_name.endswith((".jpg", ".jpeg", ".png", ".bmp")):
        image_path = os.path.join(images_folder, image_name)
        annotation_path = os.path.join(annotations_folder, image_name.replace(os.path.splitext(image_name)[1], ".txt"))

        # Charger l'image et l'annotation YOLO
        image = cv2.imread(image_path)
        with open(annotation_path, "r") as file:
            annotations = file.readlines()

        # Obtenir les dimensions de l'image
        height, width, _ = image.shape

        for rotation_angle in rotation_angles:
            # Appliquer la rotation à l'image
            rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), rotation_angle, 1)
            rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

            # Ajuster les annotations YOLO en conséquence
            rotated_annotations = []
            for annotation in annotations:
                parts = annotation.split()
                class_name = parts[0]
                x, y, bbox_width, bbox_height = map(float, parts[1:5])

                # Convertir les coordonnées YOLO en coordonnées absolues
                x_abs = int(x * width)
                y_abs = int(y * height)
                bbox_width_abs = int(bbox_width * width)
                bbox_height_abs = int(bbox_height * height)

                # Appliquer la rotation aux coordonnées du rectangle englobant
                rotated_x, rotated_y = cv2.transform(np.array([[[x_abs, y_abs]]], dtype=np.float32), rotation_matrix)[0][0]
                rotated_bbox_width, rotated_bbox_height = cv2.transform(np.array([[[bbox_width_abs, bbox_height_abs]]], dtype=np.float32), rotation_matrix)[0][0]

                # Convertir les coordonnées absolues en coordonnées YOLO
                rotated_x_yolo = rotated_x / width
                rotated_y_yolo = rotated_y / height
                rotated_bbox_width_yolo = rotated_bbox_width / width
                rotated_bbox_height_yolo = rotated_bbox_height / height

                rotated_annotations.append(f"{class_name} {rotated_x_yolo:.6f} {rotated_y_yolo:.6f} {rotated_bbox_width_yolo:.6f} {rotated_bbox_height_yolo:.6f}\n")

            # Enregistrer l'image et les nouvelles annotations
            rotated_image_path = os.path.join(output_folder, f"rotated_{rotation_angle}_{image_name}")
            rotated_annotation_path = os.path.join(output_folder, f"rotated_{rotation_angle}_{image_name.replace(os.path.splitext(image_name)[1], '.txt')}")

            cv2.imwrite(rotated_image_path, rotated_image)
            with open(rotated_annotation_path, "w") as file:
                file.writelines(rotated_annotations)

        # Copier l'image originale dans le dossier de sortie
        original_image_path = os.path.join(output_folder, f"original_{image_name}")
        cv2.imwrite(original_image_path, image)